In [3]:
install.packages("VancouvR")
install.packages("geojsonio")

library(geojsonio)
library(tidyverse)
library(dplyr)
library("knitr")

Installing package into 'C:/Users/Kaiyan Zhang/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'VancouvR' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Kaiyan Zhang\AppData\Local\Temp\RtmporPcNt\downloaded_packages


Installing package into 'C:/Users/Kaiyan Zhang/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'geojsonio' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Kaiyan Zhang\AppData\Local\Temp\RtmporPcNt\downloaded_packages


Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson


Attaching package: 'geojsonio'


The following object is masked from 'package:base':

    pretty


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [4]:
install.packages("cancensus")
library(cancensus)
options(cancensus.api_key='CensusMapper_f228791d9506a7a747ece66db73b95be')
options(cancensus.cache_path = 'C:\\Users\\Kaiyan Zhang\\Desktop\\econ326')

Installing package into 'C:/Users/Kaiyan Zhang/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'cancensus' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Kaiyan Zhang\AppData\Local\Temp\RtmporPcNt\downloaded_packages


Census data is currently stored temporarily.

 In order to speed up performance, reduce API quota usage, and reduce unnecessary network calls, please set up a persistent cache directory via `set_cancensus_cache_path('<local cache path>', install = TRUE)`.
 This will add your cache directory as environment varianble to your .Renviron to be used across sessions and projects.





In [5]:
census_data <- get_census(dataset='CA16', 
regions=list(CMA="59933",CSD="5915022"), 
vectors=c("v_CA16_4951","v_CA16_5141"), 
labels="detailed", 
geo_format='sf', level='CT')|>
as.tibble()
# |>
# filter('Region Name' == 'Vancouver')

Warning message:
"`as.tibble()` was deprecated in tibble 2.0.0.
ℹ Please use `as_tibble()` instead.
ℹ The signature and semantics have changed, see `?as_tibble`."
Reading vectors data from local cache.

Reading geo data from local cache.



In [6]:
glimpse(census_data)

Rows: 478
Columns: 16
$ `Shape Area`                                                                                                                                                                        <dbl> …
$ Type                                                                                                                                                                                <fct> …
$ Households                                                                                                                                                                          <int> …
$ `Adjusted Population (previous Census)`                                                                                                                                             <int> …
$ PR_UID                                                                                                                                                                              <chr> …
$ Dwellings                 

In [7]:
library(VancouvR)
search_cov_datasets("property-tax") %>%
  select(dataset_id,title)

dataset_id,title
<chr>,<chr>
property-tax-report-2006-2010,Property tax report 2006-2010
property-tax-report-2016-2019,Property tax report 2016-2019
property-tax-report-2011-2015,Property tax report 2011-2015
property-tax-report,Property tax report


In [8]:
tax_data <- get_cov_data(dataset_id = "property-tax-report-2016-2019",
                         where="tax_assessment_year='2016'")|>
    as.tibble()

glimpse(tax_data)


Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `current_land_value =
  .Primitive("as.integer")(current_land_value)`.
Caused by warning:
! NAs introduced by coercion to integer range"


Rows: 199,924
Columns: 29
$ pid                        <chr> "019-179-049", "028-786-394", "026-041-138"…
$ legal_type                 <chr> "STRATA", "STRATA", "STRATA", "STRATA", "ST…
$ folio                      <chr> "120605980191", "589118930004", "6031860600…
$ land_coordinate            <chr> "12060598", "58911893", "60318606", "646159…
$ zoning_district            <chr> "CD-1 (322)", "CD-1 (364)", "CD-1 (264)", "…
$ zoning_classification      <chr> "Comprehensive Development", "Comprehensive…
$ lot                        <chr> "191", "4", "53", "2", "1", "21", "31", "78…
$ plan                       <chr> "LMS1866", "EPS677", "BCS966", "VAS1717", "…
$ block                      <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ district_lot               <chr> "541", NA, "2037", "526", "526", "185", "54…
$ from_civic_number          <chr> "1601", "108", "804", "2", NA, "305", "406"…
$ to_civic_number            <chr> "989", "277", "120", "704", "1574", "1045",…
$ street_name 

In [9]:
colnames(tax_data)

[1] "pid"                        "legal_type"                
 [3] "folio"                      "land_coordinate"           
 [5] "zoning_district"            "zoning_classification"     
 [7] "lot"                        "plan"                      
 [9] "block"                      "district_lot"              
[11] "from_civic_number"          "to_civic_number"           
[13] "street_name"                "property_postal_code"      
[15] "narrative_legal_line1"      "narrative_legal_line2"     
[17] "narrative_legal_line3"      "current_land_value"        
[19] "current_improvement_value"  "tax_assessment_year"       
[21] "narrative_legal_line4"      "narrative_legal_line5"     
[23] "previous_improvement_value" "previous_land_value"       
[25] "year_built"                 "big_improvement_year"      
[27] "tax_levy"                   "neighbourhood_code"        
[29] "report_year"

In [10]:
url = "https://raw.githubusercontent.com/Irene-Berezin/econ326/refs/heads/main/crimedata_csv_AllNeighbourhoods_AllYears/crimedata_csv_AllNeighbourhoods_AllYears.csv"
intentional_crime_data <- read_csv(url)|>
    as.tibble()|>
    select(TYPE, YEAR, NEIGHBOURHOOD)|>
    rename(type = TYPE, year = YEAR, neighbourhood = NEIGHBOURHOOD)|>
    filter(!type %in% c("Vehicle Collision or Pedestrian Struck (with Fatality)",
    "Vehicle Collision or Pedestrian Struck (with Injury)",
    "Mischief"))|>
    filter(year == 2016)|>
    group_by(type)|>
    arrange(year)

glimpse(intentional_crime_data)


Rows: 895823 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 37,802
Columns: 3
Groups: type [8]
$ type          <chr> "Offence Against a Person", "Offence Against a Person", …
$ year          <dbl> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 20…
$ neighbourhood <chr> "Kitsilano", "Kitsilano", "Kitsilano", "Kitsilano", "Kit…


In [14]:
local_area_boundary <- get_cov_data(dataset_id ="local-area-boundary")|>
    as_tibble()|>
    glimpse()

Rows: 22
Columns: 4
$ name         <chr> "Arbutus Ridge", "Grandview-Woodland", "Kensington-Cedar …
$ geom         <chr> "{\"coordinates\": [[[-123.15259552001952, 49.25722503568…
$ geo_point_2d <chr> "49.24680490982789, -123.16166923820482", "49.27643960930…
$ geometry     <POLYGON [°]> POLYGON ((-123.1526 49.2572..., POLYGON ((-123.07…


In [15]:
head(local_area_boundary)

name                    
1 Arbutus Ridge           
2 Grandview-Woodland      
3 Kensington-Cedar Cottage
4 Killarney               
5 Kitsilano               
6 Strathcona              
  geom                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [16]:
property_polygons <- get_cov_data(dataset_id="property-parcel-polygons")

In [17]:
glimpse(property_polygons)

Rows: 99,944
Columns: 7
$ civic_number <chr> "1420", "7622", "7650", "7665", "7677", "1374", "7549", "…
$ streetname   <chr> "PARK DRIVE", "FRENCH ST", "GRANVILLE ST", "FRENCH ST", "…
$ tax_coord    <chr> "81613236", "13281606", "13081446", "13281631", "13281639…
$ site_id      <chr> "005243050", "011024968", "011153156", "008030430", "0073…
$ geom         <chr> "{\"coordinates\": [[[-123.13809866249942, 49.21664290375…
$ geo_point_2d <chr> "49.21644851570725, -123.13804215250474", "49.21669685474…
$ geometry     <POLYGON [°]> POLYGON ((-123.1381 49.2166..., POLYGON ((-123.13…
